# ResNet50 with DataAugmentation Modified

- Res_Net 50 
- Res-Net50 modified 
- RGB images 
- The size of the image is resized to 128x128

In [2]:
import numpy as np
from keras.models import Sequential
from keras.layers.convolutional import Conv3D
from keras.layers.convolutional_recurrent import ConvLSTM2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.layers import Dense, Activation
from keras.layers import Input
from keras.utils import multi_gpu_model
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Input
from keras.layers import Flatten,Dropout
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image 
from keras.models import Model

Using TensorFlow backend.


###### Loading data 
Intially the datas are structured to input to the convolutional layer by converting the images in to numpy array, normalized (changing the range of pixel intensity values) and reshaped accordingly. Finally stored in the path shown below.

In [3]:
x_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/x_data_all.npy')  # data path 
y_train = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/train/y_data_all.npy')

x_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/x_data_all.npy')
y_valid = np.load('/cai_lab/thomas/DWD/RGB_128_3Channel_Dataset/valid/y_data_all.npy')

print('x_train.shape:',x_train.shape,'y_train.shape:',y_train.shape)
print('x_valid.shape:',x_valid.shape,'y_valid.shape:',y_valid.shape)

x_train.shape: (17194, 128, 128, 3) y_train.shape: (17194, 9)
x_valid.shape: (2558, 128, 128, 3) y_valid.shape: (2558, 9)


In [4]:
batch_size = 24
resize = 128     # The image is resized to 128
n_class = 9
# n_frame = 5

## ResNet Model 

In [5]:
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.applications.resnet50 import ResNet50

inputs = Input (shape = (resize, resize, 3))

model_resnet50 = ResNet50(weights= 'imagenet', include_top=False, input_shape= (224,224,3))
model_resnet50.summary()

W0522 11:18:13.146104 140417602975488 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:4070: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.



Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [6]:
# Changing the fixed input of resnet50 to the reduced image size 

output_resnet50 = model_resnet50(inputs)

# Flatten the layer and add Fc layers 

y=GlobalAveragePooling2D()(output_resnet50)

dense_layer = Dense(1024, activation = 'relu')(y)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(1024, activation = 'relu')(dense_layer)
dense_layer = BatchNormalization()(dense_layer)
dense_layer = Dropout(0.5)(dense_layer)
dense_layer = Dense(512, activation = 'relu')(dense_layer)

final_output = Dense(n_class, activation = 'softmax')(dense_layer)


final_model = Model ([inputs], final_output)

final_model.summary()



Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 128, 128, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              2098176   
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096

### Model Saving and Compling 

In [7]:

save_name = 'Comp_ResNet50_RGB_128'
save_path = './checkpoints/' + save_name

if os.path.exists(save_path)==False:
    os.makedirs(save_path)

checkpoint = ModelCheckpoint(save_path +'/checkpoint-{epoch:04d}.h5', verbose=1, monitor='val_loss', save_best_only=False, mode='auto', period=1)
tensorboard = TensorBoard(log_dir='./graph/'+ save_name, histogram_freq=0, write_graph=True, write_images=True)
callback_list = [checkpoint, tensorboard]

final_model = multi_gpu_model(final_model, gpus=2)


############## Data Augmenatation ########

# aug = ImageDataGenerator(
#         rotation_range=20,
#         zoom_range=0.15,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.15,
#         horizontal_flip=True,
#         fill_mode="nearest")


# Compiling
sgd = optimizers.SGD(lr = 0.001)

final_model.compile(loss='categorical_crossentropy',optimizer = sgd,metrics=['accuracy'])


    
final_model.fit(x_train, y_train, 
                     batch_size=batch_size,
                    validation_data=(x_valid, y_valid),       
                    epochs=60,
                    verbose=1,
                    callbacks=callback_list)

# Data Augmenatation
# final_model.fit_generator(aug.flow(x_train, y_train, batch_size=batch_size),
#                     validation_data=(x_valid, y_valid),
#                     steps_per_epoch = len(x_train) // batch_size,        
#                     epochs=60,
#                     verbose=1,
#                     callbacks=callback_list)






final_model.save(save_path + '/epoch-finish.h5')
final_model.save_weights(save_path +'/epoch-finish_weights.h5')

W0522 11:18:31.092501 140417602975488 deprecation.py:323] From /home/cai/.local/lib/python3.5/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0522 11:18:42.944916 140417602975488 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.

W0522 11:18:42.971890 140417602975488 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:200: The name tf.summary.merge_all is deprecated. Please use tf.compat.v1.summary.merge_all instead.

W0522 11:18:42.972825 140417602975488 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras

Train on 17194 samples, validate on 2558 samples
Epoch 1/60
17194/17194 [==============================] - 96s 6ms/step - loss: 1.5825 - accuracy: 0.4770 - val_loss: 1.3507 - val_accuracy: 0.5465

Epoch 00001: saving model to ./checkpoints/Comp_ResNet50_RGB_128/checkpoint-0001.h5


W0522 11:20:45.795880 140417602975488 deprecation_wrapper.py:119] From /home/cai/.local/lib/python3.5/site-packages/keras/callbacks/tensorboard_v1.py:343: The name tf.Summary is deprecated. Please use tf.compat.v1.Summary instead.



Epoch 2/60
17194/17194 [==============================] - 79s 5ms/step - loss: 0.4815 - accuracy: 0.8492 - val_loss: 1.2622 - val_accuracy: 0.5461

Epoch 00002: saving model to ./checkpoints/Comp_ResNet50_RGB_128/checkpoint-0002.h5
Epoch 3/60
17194/17194 [==============================] - 79s 5ms/step - loss: 0.2365 - accuracy: 0.9315 - val_loss: 1.2327 - val_accuracy: 0.5676

Epoch 00003: saving model to ./checkpoints/Comp_ResNet50_RGB_128/checkpoint-0003.h5
Epoch 4/60
17194/17194 [==============================] - 79s 5ms/step - loss: 0.1635 - accuracy: 0.9524 - val_loss: 1.2163 - val_accuracy: 0.5813

Epoch 00004: saving model to ./checkpoints/Comp_ResNet50_RGB_128/checkpoint-0004.h5
Epoch 5/60
17194/17194 [==============================] - 79s 5ms/step - loss: 0.1087 - accuracy: 0.9692 - val_loss: 1.1837 - val_accuracy: 0.6099

Epoch 00005: saving model to ./checkpoints/Comp_ResNet50_RGB_128/checkpoint-0005.h5
Epoch 6/60
17194/17194 [==============================] - 79s 5ms/step -